# Programming Assignment #2: Databases

In this assignment, we will learn a little bit about databases by implementing some relational algebra operators.

## Databases

Databases store data in a collection of **tables** or **relations**, which you can think of as Excel spreadsheets. 
For example, here is how some facts about some (hypothetical) students:

| StudentId | FirstName | LastName | GPA |
|:----------|:----------|:---------|----:|
| W003      | Richard   | Baslim   | 2.7 |
| W001      | Maureen   | Johnson  | 4.0 |
| W012      | Lazarus   | Long     | 3.4 |
| W014      | Lummox    | Lummox   | 2.1 |
| W010      | Hazel     | Stone    | 2.9 |
| W006      | Dejah     | Thoris   | 3.4 |

> Note: What does this have to do with **relations** as discussed in class? It turns out that **relational databases** are all about relations! The first table above is a relation, in the sense that it is a subset of the cross product of StudentIds, FirstNames, LastNames, and GPAs. Those are names, but really what we mean is the crossproduct of the possible types: alphanum x alpha x alpha x rational.

An important aspect of relational databases is the notion of a **schema**, which describes the names of the tables, columns, and the types of the columns. For example, the table above may be called STUDENTS, the column names are listed in the headings, and the column types are described in the note above. There is more to schemas, but we'll ignore that in this assignment.

## TODO-0 (no points): Data Definitions and Sample Tables

Here is how we can represent that table, and some others, in ACL2. Note how the type information in the schema can be described using `defdata`. (We ignore the column names here and use column indexes instead.) Note how we define detailed schema information for each table and also give a "generic" table definition; the operators we'll be defining work on generic tables. There's nothing to do here, but be sure to execute this cell to initialize the data we'll be using later.

In [178]:
(defsnapshot from-the-top)
(defconst *STUDENTS*
    '((W003   Richard   Baslim   2.7)
      (W001   Maureen   Johnson  4.0)
      (W012   Lazarus   Long     3.4)
      (W014   Lummox    Lummox   2.1)
      (W010   Hazel     Stone    2.9)
      (W006   Dejah     Thoris   3.4)))
      
(defdata student-row
    (list symbol symbol symbol rational))
(defdata student-table
    (listof :student-row))
(check-true (student-tablep *STUDENTS*))

(defconst *COURSES*
    '((COSC2300     discrete-structures "Discrete mathematics for computer science")
      (COSC4820     databases           "Database design and implementation")
      (ENGL3666     science-fiction     "Classic works from the science fiction canon")))

(defdata courses-row
    (list :symbol :symbol :string))
(defdata courses-table
    (listof :courses-row))
(check-true (courses-tablep *COURSES*))

(defconst *TRANSCRIPT*
    '((W001   COSC2300     A)
      (W001   COSC4820     A)
      (W001   ENGL3666     A)
      (W012   COSC2300     A)
      (W012   ENGL3666     B)
      (W010   COSC2300     B)
      (W010   ENGL3666     A)))

(defdata transcript-row
    (list symbol symbol symbol))
(defdata transcript-table
    (listof transcript-row))
(check-true (transcript-tablep *TRANSCRIPT*))

(defdata generic-value (oneof :symbol :string :rational))
(defdata generic-row
    (listof :generic-value))
(defdata generic-table
    (listof :generic-row))
(check-true (generic-tablep *STUDENTS*))
(check-true (generic-tablep *COURSES*))
(check-true (generic-tablep *TRANSCRIPT*))


ACL2S !>>(DEFSNAPSHOT FROM-THE-TOP)
          20:x(DEFMACRO DEFSNAPSHOT (LABEL) ...)

Summary
Form:  ( DEFLABEL FROM-THE-TOP ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 FROM-THE-TOP
ACL2S !>>(DEFCONST *STUDENTS*
                   '((W003 RICHARD BASLIM 27/10)
                     (W001 MAUREEN JOHNSON 4)
                     (W012 LAZARUS LONG 17/5)
                     (W014 LUMMOX LUMMOX 21/10)
                     (W010 HAZEL STONE 29/10)
                     (W006 DEJAH THORIS 17/5)))

Summary
Form:  ( DEFCONST *STUDENTS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *STUDENTS*
ACL2S !>>(DEFDATA STUDENT-ROW
                  (LIST SYMBOL SYMBOL SYMBOL RATIONAL))
 Predicate events...
Form:  ( DEFUN STUDENT-ROWP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
 (STUDENT-ROWP ACL2::V1) <= body -

## TODO-1 (10 pts): Projection


Let's start with **projection**. These is one of the "relational algebra" operators, and it returns a new table that has only some of the columns from the input table. For example, the result of selecting columns 1 and 3 from
the STUDENTS table above should be:

| FirstName | GPA |
|:----------|----:|
| Richard   | 2.7 |
| Maureen   | 4.0 |
| Lazarus   | 3.4 |
| Lummox    | 2.1 |
| Hazel     | 2.9 |
| Dejah     | 3.4 |

Note that we are using 0-based indexing here, as is traditional in computer science.

Your first task is to write the function `(projection indexes relation)` that performs this operation. For example, the projection above corresponds to `(projection '(1 3) *STUDENTS*)`.

> Hint: You may find it helpful to write **two** functions, the `projection` function and a helper function that projects a single row.

> Hint: Play around with the built-in function `nth`, as in `(nth 3 '(1 2 3 4 5))`. It will help you to write this function.

In [179]:
(defsnapshot todo-1)
(nth 0 '())
(definec helper (indexes :nat-list row :generic-row) :generic-row
    (if (or (endp indexes) (> (+ (first indexes) 1) (len row)))
        nil
        (append (list (nth (first indexes) row)) (helper (rest indexes) row))
    )
) 
(definec projection (indexes :nat-list relation :generic-table) :generic-table
    (if (endp relation)
        '()
        (append (list (helper indexes (first relation))) (projection indexes (rest relation)))
    )  
)

(check-expect (projection '(1 3) *STUDENTS*) '((RICHARD 2.7) (MAUREEN 4.0) (LAZARUS 3.4) (LUMMOX 2.1) (HAZEL 2.9) (DEJAH 3.4)))
(check-expect (projection '(0 1) *COURSES*) '((COSC2300 DISCRETE-STRUCTURES) (COSC4820 DATABASES) (ENGL3666 SCIENCE-FICTION)))
(check-expect (projection '(1 2) *TRANSCRIPT*) '((COSC2300 A) (COSC4820 A) (ENGL3666 A) (COSC2300 A) (ENGL3666 B) (COSC2300 B) (ENGL3666 A)))
(check-expect (projection '(0 2) *COURSES*) '((COSC2300 "Discrete mathematics for computer science") (COSC4820 "Database design and implementation") (ENGL3666 "Classic works from the science fiction canon")))
(check-expect (projection '(0 2) *STUDENTS*) '((W003 BASLIM) (W001 JOHNSON) (W012 LONG) (W014 LUMMOX) (W010 STONE) (W006 THORIS)))


ACL2S !>>(DEFSNAPSHOT TODO-1)

Summary
Form:  ( DEFLABEL TODO-1 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-1
ACL2S !>>(NTH 0 'NIL)
NIL
ACL2S !>>(DEFINEC HELPER
                  (INDEXES NAT-LIST ROW GENERIC-ROW)
                  GENERIC-ROW
                  (IF (OR (ENDP INDEXES)
                          (> (+ (FIRST INDEXES) 1) (LEN ROW)))
                      NIL
                      (APPEND (LIST (NTH (FIRST INDEXES) ROW))
                              (HELPER (REST INDEXES) ROW))))

Form:  ( TEST-DEFINITION HELPER ... )
Form:  ( TEST-BODY-CONTRACTS HELPER... ) 
Form:  ( TEST-FUNCTION-CONTRACT HELPER ...) 
Testing: Done 
Elapsed Run Time: 1.46 seconds
Form:  ( ADMIT-DEFINITION HELPER ... )
Time:  0.04 seconds (prove: 0.00, print: 0.00, other: 0.04)
Form:  ( PROVE-FUNCTION-CONTRACT HELPER ... )
Time:  1.67 seconds (prove: 0.73, print: 0.00, other: 0.94)
Form:  ( PROVE-BODY-CONTRACTS HELPER ... )
Time:  0.00 seconds (prove: 0.00, print: 0.0

## TODO-2 (10 pts): Selection, Part 1

If projection lets you keep some of the columns of a table, **selection** is all about keeping some rows of the table and discarding the rest. We'll start with the simplest version of selection, and that is to select the rows that have a given value in one of their columns, e.g., the rows with gpa (i.e., column #3) equal to 3.4:

| StudentId | FirstName | LastName | GPA |
|:----------|:----------|:---------|----:|
| W012      | Lazarus   | Long     | 3.4 |
| W006      | Dejah     | Thoris   | 3.4 |

Implement the function `(selection-constant index value relation)` that selects the rows in relation where the column given by the index is equal to the value provided. E.g., the call `(selection-constant 3 3.4 *STUDENTS*)` generates the results above.

> Hint: Remember that you can always define auxiliary functions. Also, the hints to TODO-1 apply here as well.

> Hint: Use the `generic-` types defined in TODO-0.

In [180]:
(defsnapshot todo-2)
(definec selection-constant (index :nat value :generic-value relation :generic-table) :generic-table
    (if (or (endp relation) (equal value nil))
        '()
        (if (equal (nth index (first relation)) value)
            (append (list (first relation)) (selection-constant index value (rest relation)))
            (append nil (selection-constant index value (rest relation)))
        )
    ) 
)

(selection-constant 3 nil *STUDENTS*)


(check-expect (selection-constant 3 3.4 *STUDENTS*) '((W012 LAZARUS LONG 17/5) (W006 DEJAH THORIS 17/5)))
(check-expect (selection-constant 0 'W003 *STUDENTS*) '((W003 RICHARD BASLIM 27/10)))
(check-expect (selection-constant 0 'W001 *TRANSCRIPT*) '((W001 COSC2300 A) (W001 COSC4820 A) (W001 ENGL3666 A)))
(check-expect (selection-constant 1 'COSC2300 *TRANSCRIPT*) '((W001 COSC2300 A) (W012 COSC2300 A) (W010 COSC2300 B)))
(check-expect (selection-constant 2 'A *TRANSCRIPT*) '((W001 COSC2300 A) (W001 COSC4820 A) (W001 ENGL3666 A) (W012 COSC2300 A) (W010 ENGL3666 A)))

ACL2S !>>(DEFSNAPSHOT TODO-2)

Summary
Form:  ( DEFLABEL TODO-2 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-2
ACL2S !>>(DEFINEC
          SELECTION-CONSTANT
          (INDEX NAT VALUE
                 GENERIC-VALUE RELATION GENERIC-TABLE)
          GENERIC-TABLE
          (IF
             (OR (ENDP RELATION) (EQUAL VALUE NIL))
             'NIL
             (IF (EQUAL (NTH INDEX (FIRST RELATION))
                        VALUE)
                 (APPEND (LIST (FIRST RELATION))
                         (SELECTION-CONSTANT INDEX VALUE (REST RELATION)))
                 (APPEND NIL
                         (SELECTION-CONSTANT INDEX VALUE (REST RELATION))))))

Form:  ( TEST-DEFINITION SELECTION-CONSTANT ... )
Form:  ( TEST-BODY-CONTRACTS SELECTION-CONSTANT... ) 
Form:  ( TEST-FUNCTION-CONTRACT SELECTION-CONSTANT ...) 
Testing: Done 
Elapsed Run Time: 2.07 seconds
Form:  ( ADMIT-DEFINITION SELECTION-CONSTANT ... )
Time:  0.02 seconds (prove: 0.00, print: 

## TODO-3 (10 pts): Selection, Part 2

Next, let's look at a different type of selection. Here, we will select the rows where one column has the same value as another column. For example, we can select the rows where the FirstName and LastName have the same value:

| StudentId | FirstName | LastName | GPA |
|:----------|:----------|:---------|----:|
| W014      | Lummox    | Lummox   | 2.1 |

Implement the function `(selection-column index1 index2 relation)` that selects the rows in relation where the two columns have the same value. E.g., the call `(selection-column 1 2 *STUDENTS*)` generates the results above.

> Note: You may not think this particular operation is useful, but just wait a couple of TODOs. It turns out to be part of something very important in databases, called the **join** operator.

In [181]:
(defsnapshot todo-3)

(definec selection-column (index1 :nat index2 :nat relation :generic-table) :generic-table
    (if (endp relation)
        '()
        (if (equal (nth index1 (first relation)) (nth index2 (first relation)))
            (append (list (first relation)) (selection-column index1 index2 (rest relation)))
            (append nil (selection-column index1 index2 (rest relation)))
        )
    )
)

(check-expect (selection-column 1 2 *STUDENTS*) '((W014 LUMMOX LUMMOX 21/10)))
(check-expect (selection-column 0 1 *COURSES*) nil)
(check-expect (selection-column 0 0 *STUDENTS*) '((W003 RICHARD BASLIM 27/10) (W001 MAUREEN JOHNSON 4) (W012 LAZARUS LONG 17/5) (W014 LUMMOX LUMMOX 21/10) (W010 HAZEL STONE 29/10) (W006 DEJAH THORIS 17/5)))
(check-expect (selection-column 2 1 *STUDENTS*) '((W014 LUMMOX LUMMOX 21/10)))
(check-expect (selection-column 1 0 *TRANSCRIPT*) nil)

ACL2S !>>(DEFSNAPSHOT TODO-3)

Summary
Form:  ( DEFLABEL TODO-3 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-3
ACL2S !>>(DEFINEC
          SELECTION-COLUMN
          (INDEX1 NAT INDEX2 NAT RELATION GENERIC-TABLE)
          GENERIC-TABLE
          (IF
             (ENDP RELATION)
             'NIL
             (IF (EQUAL (NTH INDEX1 (FIRST RELATION))
                        (NTH INDEX2 (FIRST RELATION)))
                 (APPEND (LIST (FIRST RELATION))
                         (SELECTION-COLUMN INDEX1 INDEX2 (REST RELATION)))
                 (APPEND NIL
                         (SELECTION-COLUMN INDEX1 INDEX2 (REST RELATION))))))

Form:  ( TEST-DEFINITION SELECTION-COLUMN ... )
Form:  ( TEST-BODY-CONTRACTS SELECTION-COLUMN... ) 
Form:  ( TEST-FUNCTION-CONTRACT SELECTION-COLUMN ...) 
Testing: Done 
Elapsed Run Time: 2.40 seconds
Form:  ( ADMIT-DEFINITION SELECTION-COLUMN ... )
Time:  0.03 seconds (prove: 0.00, print: 0.00, other: 0.03)
Form:  ( PROVE

## TODO-4 (10 pts): Using Relational Algebra

Using the **functions you already defined**, write a `check-expect` expression that finds the first name and last name (only) of anybody who has a GPA of exactly 3.4. This should give you an idea of how relational algebra works in practice. In fact, here is one version of the expression I'm asking you to implement, but in relational algebra  syntax:

$$\pi_{FirstName, LastName}(\sigma_{GPA=3.4}(STUDENTS))$$

> Curiosity: The Greek letters $\pi$ and $\sigma$ stand for **p**rojection and **s**election, respectively.

> Note: This question is just meant to give you an idea of how relational algebra is used to **query** databases, so you can get information from the facts stored in the database.

In [182]:
(defsnapshot todo-4)

(check-expect (projection '(1 2 3) (selection-constant 3 3.4 *STUDENTS*)) '((LAZARUS LONG 17/5) (DEJAH THORIS 17/5)))
(check-expect (projection '(0) (selection-constant 1 'databases *COURSES*)) '((COSC4820)))
(check-expect (projection '(0 1 2) (selection-constant 1 'COSC2300 *TRANSCRIPT*)) '((W001 COSC2300 A) (W012 COSC2300 A) (W010 COSC2300 B)))
(check-expect (projection '() (selection-constant 3 3.4 *STUDENTS*)) '(nil nil))
(check-expect (projection '(0 2) (selection-constant 2 'A *TRANSCRIPT*)) '((W001 A) (W001 A) (W001 A) (W012 A) (W010 A)))



ACL2S !>>(DEFSNAPSHOT TODO-4)

Summary
Form:  ( DEFLABEL TODO-4 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-4
ACL2S !>>(CHECK-EXPECT (PROJECTION '(1 2 3)
                                   (SELECTION-CONSTANT 3 17/5 *STUDENTS*))
                       '((LAZARUS LONG 17/5)
                         (DEJAH THORIS 17/5)))
 :PASSED
ACL2S !>>(CHECK-EXPECT (PROJECTION '(0)
                                   (SELECTION-CONSTANT 1 'DATABASES
                                                       *COURSES*))
                       '((COSC4820)))
 :PASSED
ACL2S !>>(CHECK-EXPECT (PROJECTION '(0 1 2)
                                   (SELECTION-CONSTANT 1 'COSC2300
                                                       *TRANSCRIPT*))
                       '((W001 COSC2300 A)
                         (W012 COSC2300 A)
                         (W010 COSC2300 B)))
 :PASSED
ACL2S !>>(CHECK-EXPECT (PROJECTION 'NIL
                                   (SELECTION-CON

## TODO-5 (10 pts): Cross Product

Now let's implement another important operation, and the first that involves more than one table. The cross-product is exactly the same as defined in class. The cross-product of table `A` with `m` columns and `B` with `n` columns is a table with `m+n` columns where the first `m` columns come from `A` and the last `n` columns come from `B`. For example, the cross product of STUDENTS and COURSES is

| StudentId | FirstName | LastName | GPA | CourseId   | Title               | Description                                |
|:----------|:----------|:---------|----:|:-----------|:--------------------|:-------------------------------------------------|
| W003      | Richard   | Baslim   | 2.7 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W003      | Richard   | Baslim   | 2.7 | COSC4820   | databases           | "Database design and implementation"           |
| W003      | Richard   | Baslim   | 2.7 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |
| W001      | Maureen   | Johnson  | 4.0 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W001      | Maureen   | Johnson  | 4.0 | COSC4820   | databases           | "Database design and implementation"           |
| W001      | Maureen   | Johnson  | 4.0 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |
| W012      | Lazarus   | Long     | 3.4 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W012      | Lazarus   | Long     | 3.4 | COSC4820   | databases           | "Database design and implementation"           |
| W012      | Lazarus   | Long     | 3.4 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |
| W014      | Lummox    | Lummox   | 2.1 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W014      | Lummox    | Lummox   | 2.1 | COSC4820   | databases           | "Database design and implementation"           |
| W014      | Lummox    | Lummox   | 2.1 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |
| W010      | Hazel     | Stone    | 2.9 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W010      | Hazel     | Stone    | 2.9 | COSC4820   | databases           | "Database design and implementation"           |
| W010      | Hazel     | Stone    | 2.9 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |
| W006      | Dejah     | Thoris   | 3.4 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W006      | Dejah     | Thoris   | 3.4 | COSC4820   | databases           | "Database design and implementation"           |
| W006      | Dejah     | Thoris   | 3.4 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |

Write the function `(cross relation1 relation2)` that returns the cross-product of two relations, e.g., `(cross *STUDENTS* *COURSES*)`.

In [183]:
(defsnapshot todo-5)
(definec helper2 (relation1 :generic-row relation2 :generic-table) :generic-table
    (if (endp relation2)
        '()
        (if (endp relation1)
            '()
             (if (endp (first relation2))
                (append nil (helper2 relation1 (rest relation2)))
                (append (list (append relation1 (first relation2))) (helper2 relation1 (rest relation2)))  
            )
        )
    )
)

(definec cross (relation1 :generic-table relation2 :generic-table) :generic-table
    (if (endp relation1)
        '()
        (append (helper2 (first relation1) relation2) (cross (rest relation1) relation2))
    ) 
)


(check-expect (cross *STUDENTS* *COURSES*) '((W003 RICHARD BASLIM
       27/10 COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W003 RICHARD BASLIM 27/10 COSC4820 DATABASES
       "Database design and implementation")
 (W003 RICHARD
       BASLIM 27/10 ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W001 MAUREEN
       JOHNSON 4 COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W001 MAUREEN JOHNSON 4 COSC4820 DATABASES
       "Database design and implementation")
 (W001 MAUREEN
       JOHNSON 4 ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W012 LAZARUS
       LONG 17/5 COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W012 LAZARUS LONG 17/5 COSC4820 DATABASES
       "Database design and implementation")
 (W012 LAZARUS
       LONG 17/5 ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W014 LUMMOX LUMMOX
       21/10 COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W014 LUMMOX LUMMOX 21/10 COSC4820 DATABASES
       "Database design and implementation")
 (W014 LUMMOX
       LUMMOX 21/10 ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W010 HAZEL
       STONE 29/10 COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W010 HAZEL STONE 29/10 COSC4820 DATABASES
       "Database design and implementation")
 (W010 HAZEL
       STONE 29/10 ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W006 DEJAH
       THORIS 17/5 COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W006 DEJAH THORIS 17/5 COSC4820 DATABASES
       "Database design and implementation")
 (W006 DEJAH
       THORIS 17/5 ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon"))) 

(check-expect (cross *COURSES* *COURSES*) '((COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science")
 (COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           COSC4820 DATABASES
           "Database design and implementation")
 (COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon")
 (COSC4820 DATABASES
           "Database design and implementation"
           COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science")
 (COSC4820 DATABASES
           "Database design and implementation"
           COSC4820 DATABASES
           "Database design and implementation")
 (COSC4820 DATABASES
           "Database design and implementation"
           ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon")
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science")
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           COSC4820 DATABASES
           "Database design and implementation")
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon")))

(check-expect (cross *TRANSCRIPT* *COURSES*) '((W001 COSC2300 A COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W001 COSC2300 A COSC4820 DATABASES
       "Database design and implementation")
 (W001 COSC2300 A ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W001 COSC4820 A COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W001 COSC4820 A COSC4820 DATABASES
       "Database design and implementation")
 (W001 COSC4820 A ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W001 ENGL3666 A COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W001 ENGL3666 A COSC4820 DATABASES
       "Database design and implementation")
 (W001 ENGL3666 A ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W012 COSC2300 A COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W012 COSC2300 A COSC4820 DATABASES
       "Database design and implementation")
 (W012 COSC2300 A ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W012 ENGL3666 B COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W012 ENGL3666 B COSC4820 DATABASES
       "Database design and implementation")
 (W012 ENGL3666 B ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W010 COSC2300 B COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W010 COSC2300 B COSC4820 DATABASES
       "Database design and implementation")
 (W010 COSC2300 B ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")
 (W010 ENGL3666 A COSC2300 DISCRETE-STRUCTURES
       "Discrete mathematics for computer science")
 (W010 ENGL3666 A COSC4820 DATABASES
       "Database design and implementation")
 (W010 ENGL3666 A ENGL3666 SCIENCE-FICTION
       "Classic works from the science fiction canon")))

(check-expect (cross *COURSES* *STUDENTS*) '((COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           W003 RICHARD BASLIM 27/10)
 (COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           W001 MAUREEN JOHNSON 4)
 (COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           W012 LAZARUS LONG 17/5)
 (COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           W014 LUMMOX LUMMOX 21/10)
 (COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           W010 HAZEL STONE 29/10)
 (COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           W006 DEJAH THORIS 17/5)
 (COSC4820 DATABASES
           "Database design and implementation"
           W003 RICHARD BASLIM 27/10)
 (COSC4820 DATABASES
           "Database design and implementation"
           W001 MAUREEN JOHNSON 4)
 (COSC4820 DATABASES
           "Database design and implementation"
           W012 LAZARUS LONG 17/5)
 (COSC4820 DATABASES
           "Database design and implementation"
           W014 LUMMOX LUMMOX 21/10)
 (COSC4820 DATABASES
           "Database design and implementation"
           W010 HAZEL STONE 29/10)
 (COSC4820 DATABASES
           "Database design and implementation"
           W006 DEJAH THORIS 17/5)
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           W003 RICHARD BASLIM 27/10)
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           W001 MAUREEN JOHNSON 4)
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           W012 LAZARUS LONG 17/5)
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           W014 LUMMOX LUMMOX 21/10)
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           W010 HAZEL STONE 29/10)
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           W006 DEJAH THORIS 17/5)))

(check-expect (cross *STUDENTS* *TRANSCRIPT*) '((W003 RICHARD BASLIM 27/10 W001 COSC2300 A)
 (W003 RICHARD BASLIM 27/10 W001 COSC4820 A)
 (W003 RICHARD BASLIM 27/10 W001 ENGL3666 A)
 (W003 RICHARD BASLIM 27/10 W012 COSC2300 A)
 (W003 RICHARD BASLIM 27/10 W012 ENGL3666 B)
 (W003 RICHARD BASLIM 27/10 W010 COSC2300 B)
 (W003 RICHARD BASLIM 27/10 W010 ENGL3666 A)
 (W001 MAUREEN JOHNSON 4 W001 COSC2300 A)
 (W001 MAUREEN JOHNSON 4 W001 COSC4820 A)
 (W001 MAUREEN JOHNSON 4 W001 ENGL3666 A)
 (W001 MAUREEN JOHNSON 4 W012 COSC2300 A)
 (W001 MAUREEN JOHNSON 4 W012 ENGL3666 B)
 (W001 MAUREEN JOHNSON 4 W010 COSC2300 B)
 (W001 MAUREEN JOHNSON 4 W010 ENGL3666 A)
 (W012 LAZARUS LONG 17/5 W001 COSC2300 A)
 (W012 LAZARUS LONG 17/5 W001 COSC4820 A)
 (W012 LAZARUS LONG 17/5 W001 ENGL3666 A)
 (W012 LAZARUS LONG 17/5 W012 COSC2300 A)
 (W012 LAZARUS LONG 17/5 W012 ENGL3666 B)
 (W012 LAZARUS LONG 17/5 W010 COSC2300 B)
 (W012 LAZARUS LONG 17/5 W010 ENGL3666 A)
 (W014 LUMMOX LUMMOX 21/10 W001 COSC2300 A)
 (W014 LUMMOX LUMMOX 21/10 W001 COSC4820 A)
 (W014 LUMMOX LUMMOX 21/10 W001 ENGL3666 A)
 (W014 LUMMOX LUMMOX 21/10 W012 COSC2300 A)
 (W014 LUMMOX LUMMOX 21/10 W012 ENGL3666 B)
 (W014 LUMMOX LUMMOX 21/10 W010 COSC2300 B)
 (W014 LUMMOX LUMMOX 21/10 W010 ENGL3666 A)
 (W010 HAZEL STONE 29/10 W001 COSC2300 A)
 (W010 HAZEL STONE 29/10 W001 COSC4820 A)
 (W010 HAZEL STONE 29/10 W001 ENGL3666 A)
 (W010 HAZEL STONE 29/10 W012 COSC2300 A)
 (W010 HAZEL STONE 29/10 W012 ENGL3666 B)
 (W010 HAZEL STONE 29/10 W010 COSC2300 B)
 (W010 HAZEL STONE 29/10 W010 ENGL3666 A)
 (W006 DEJAH THORIS 17/5 W001 COSC2300 A)
 (W006 DEJAH THORIS 17/5 W001 COSC4820 A)
 (W006 DEJAH THORIS 17/5 W001 ENGL3666 A)
 (W006 DEJAH THORIS 17/5 W012 COSC2300 A)
 (W006 DEJAH THORIS 17/5 W012 ENGL3666 B)
 (W006 DEJAH THORIS 17/5 W010 COSC2300 B)
 (W006 DEJAH THORIS 17/5 W010 ENGL3666 A)))


ACL2S !>>(DEFSNAPSHOT TODO-5)

Summary
Form:  ( DEFLABEL TODO-5 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-5
ACL2S !>>(DEFINEC
              HELPER2
              (RELATION1 GENERIC-ROW RELATION2 GENERIC-TABLE)
              GENERIC-TABLE
              (IF (ENDP RELATION2)
                  'NIL
                  (IF (ENDP RELATION1)
                      'NIL
                      (IF (ENDP (FIRST RELATION2))
                          (APPEND NIL
                                  (HELPER2 RELATION1 (REST RELATION2)))
                          (APPEND (LIST (APPEND RELATION1 (FIRST RELATION2)))
                                  (HELPER2 RELATION1 (REST RELATION2)))))))

Form:  ( TEST-DEFINITION HELPER2 ... )
Form:  ( TEST-BODY-CONTRACTS HELPER2... ) 
Form:  ( TEST-FUNCTION-CONTRACT HELPER2 ...) 
Testing: Done 
Elapsed Run Time: 1.05 seconds
Form:  ( ADMIT-DEFINITION HELPER2 ... )
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  (

## TODO-6 (10 pts): Join

Finally, we can implement **join**, the most important relational algebra operation of all. In fact, much research into databases has gone into finding ways to implement **join** efficiently.

A join is nothing more than a cross product followed by a selection, and the selection says that one column of the first relation is equal to some column in the second relation. For example, we can join STUDENTS and TRANSCRIPT on the first column of both tables, to get

| StudentId | FirstName | LastName | GPA | StudentId | CourseId   | Grade |
|:----------|:----------|:---------|----:|:----------|:-----------|:------|
| W001      | Maureen   | Johnson  | 4.0 | W001      | COSC2300   | A     |
| W001      | Maureen   | Johnson  | 4.0 | W001      | COSC4820   | A     |
| W001      | Maureen   | Johnson  | 4.0 | W001      | ENGL3666   | A     |
| W012      | Lazarus   | Long     | 3.4 | W012      | COSC2300   | A     |
| W012      | Lazarus   | Long     | 3.4 | W012      | ENGL3666   | B     |
| W010      | Hazel     | Stone    | 2.9 | W010      | COSC2300   | B     |
| W010      | Hazel     | Stone    | 2.9 | W010      | ENGL3666   | A     |

In relational algebra notation, we would write this as $STUDENTS \bowtie_{StudentId,StudentId} TRANSCRIPT$.
Notice that this is starting to give us very useful information! We have students (with names) and the courses they've taken. If you do another join, you can get the student names **and** the course names. (See TODO-7.)

Using the functions you've already defined, implement the function `(join index1 index2 relation1 relation2)` that joins the two relations on the columns specified. E.g., the table above is the result of `(join 0 0 *STUDENTS* *TRANSCRIPT*)`.


In [184]:
(defsnapshot todo-6)
(definec join (index1 :nat index2 :nat relation1 :generic-table relation2 :generic-table) :generic-table
    (if (or (endp relation1) (endp relation2) (> index1 (+ (len relation1) (len relation2))) (> index2 (+ (len relation1) (len relation2)))) 
        '()
         (selection-column index1 (+ index2 (len (first relation1))) (cross relation1 relation2))
    )
)

(check-expect (join 0 0 *STUDENTS* *TRANSCRIPT*) '((W001 MAUREEN JOHNSON 4 W001 COSC2300 A)
 (W001 MAUREEN JOHNSON 4 W001 COSC4820 A)
 (W001 MAUREEN JOHNSON 4 W001 ENGL3666 A)
 (W012 LAZARUS LONG 17/5 W012 COSC2300 A)
 (W012 LAZARUS LONG 17/5 W012 ENGL3666 B)
 (W010 HAZEL STONE 29/10 W010 COSC2300 B)
 (W010 HAZEL STONE 29/10 W010 ENGL3666 A)))

(check-expect (join 0 1 *COURSES* *TRANSCRIPT*) '((COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           W001 COSC2300 A)
 (COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           W012 COSC2300 A)
 (COSC2300 DISCRETE-STRUCTURES
           "Discrete mathematics for computer science"
           W010 COSC2300 B)
 (COSC4820 DATABASES
           "Database design and implementation"
           W001 COSC4820 A)
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           W001 ENGL3666 A)
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           W012 ENGL3666 B)
 (ENGL3666 SCIENCE-FICTION
           "Classic works from the science fiction canon"
           W010 ENGL3666 A)))
              
(check-expect (join 0 0 *STUDENTS* *STUDENTS*) '((W003 RICHARD
       BASLIM 27/10 W003 RICHARD BASLIM 27/10)
 (W001 MAUREEN
       JOHNSON 4 W001 MAUREEN JOHNSON 4)
 (W012 LAZARUS
       LONG 17/5 W012 LAZARUS LONG 17/5)
 (W014 LUMMOX
       LUMMOX 21/10 W014 LUMMOX LUMMOX 21/10)
 (W010 HAZEL
       STONE 29/10 W010 HAZEL STONE 29/10)
 (W006 DEJAH
       THORIS 17/5 W006 DEJAH THORIS 17/5)))
                            
(check-expect (join 0 0 *TRANSCRIPT* *STUDENTS*) '((W001 COSC2300 A W001 MAUREEN JOHNSON 4)
 (W001 COSC4820 A W001 MAUREEN JOHNSON 4)
 (W001 ENGL3666 A W001 MAUREEN JOHNSON 4)
 (W012 COSC2300 A W012 LAZARUS LONG 17/5)
 (W012 ENGL3666 B W012 LAZARUS LONG 17/5)
 (W010 COSC2300 B W010 HAZEL STONE 29/10)
 (W010 ENGL3666 A W010 HAZEL STONE 29/10)))
                                          
(check-expect (join 0 1 *STUDENTS* *TRANSCRIPT*) '())


ACL2S !>>(DEFSNAPSHOT TODO-6)

Summary
Form:  ( DEFLABEL TODO-6 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-6
ACL2S !>>(DEFINEC JOIN
                  (INDEX1 NAT INDEX2 NAT RELATION1
                          GENERIC-TABLE RELATION2 GENERIC-TABLE)
                  GENERIC-TABLE
                  (IF (OR (ENDP RELATION1)
                          (ENDP RELATION2)
                          (> INDEX1
                             (+ (LEN RELATION1) (LEN RELATION2)))
                          (> INDEX2
                             (+ (LEN RELATION1) (LEN RELATION2))))
                      'NIL
                      (SELECTION-COLUMN INDEX1
                                        (+ INDEX2 (LEN (FIRST RELATION1)))
                                        (CROSS RELATION1 RELATION2))))

Form:  ( TEST-DEFINITION JOIN ... )
Form:  ( TEST-BODY-CONTRACTS JOIN... ) 
Form:  ( TEST-FUNCTION-CONTRACT JOIN ...) 
Testing: Done 
Elapsed Run Time: 3.79 seconds
Form

## TODO-7 (10 pts): Another Sample Query

Similar to TODO04, this is another exercise in writing relational algebra expressions to query a database. Using the functions already defined, use `check-expect` to find the course ID, course title, and grade of all the courses that the student with first name Lazarus and last name Long has taken. Again, here is a way of doing this using relational algebra syntax:

$$\pi_{CourseId, Title, Grade}(\sigma_{FirstName,Lazarus}(\sigma_{LastName,Long}((STUDENTS \bowtie_{StudentId,StrudentId} (TRANSCRIPT \bowtie_{CourseId,CourseId} COURSES)))))$$

In [188]:
(defsnapshot todo-7)

(check-expect (projection '(8 7 6) (join 5 0 (join 0 0 (selection-constant 2 'Long (selection-constant 1 'Lazarus *STUDENTS*)) *TRANSCRIPT*) *COURSES*)) '((DISCRETE-STRUCTURES COSC2300 A)
 (SCIENCE-FICTION ENGL3666 B)))
(check-expect (projection '(8 7 6) (join 5 0 (join 0 0 (selection-constant 2 'Lummox (selection-constant 1 'Lummox *STUDENTS*)) *TRANSCRIPT*) *COURSES*)) nil)
(check-expect (projection '(8 7 6) (join 5 0 (join 0 0 (selection-constant 2 'Johnson (selection-constant 1 'Maureen *STUDENTS*)) *TRANSCRIPT*) *COURSES*)) '((DISCRETE-STRUCTURES COSC2300 A)
 (DATABASES COSC4820 A)
 (SCIENCE-FICTION ENGL3666 A)))
(check-expect (projection '(8 7 6) (join 5 0 (join 0 0 (selection-constant 2 'Stone (selection-constant 1 'Hazel *STUDENTS*)) *TRANSCRIPT*) *COURSES*)) '((DISCRETE-STRUCTURES COSC2300 B)
 (SCIENCE-FICTION ENGL3666 A)))
(check-expect (projection '(8 7 6) (join 5 0 (join 0 0 (selection-constant 2 'Harris (selection-constant 1 'Buck *STUDENTS*)) *TRANSCRIPT*) *COURSES*)) nil)










ACL2S !>>(DEFSNAPSHOT TODO-7)
   d      46:x(DEFINEC JOIN (INDEX1 NAT INDEX2 ...)
                       ...)

Summary
Form:  ( DEFLABEL TODO-7 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-7
ACL2S !>>(CHECK-EXPECT
          (PROJECTION
             '(8 7 6)
             (JOIN 5 0
                   (JOIN 0 0
                         (SELECTION-CONSTANT 2 'LONG
                                             (SELECTION-CONSTANT 1 'LAZARUS
                                                                 *STUDENTS*))
                         *TRANSCRIPT*)
                   *COURSES*))
          '((DISCRETE-STRUCTURES COSC2300 A)
            (SCIENCE-FICTION ENGL3666 B)))
 :PASSED
ACL2S !>>(CHECK-EXPECT
          (PROJECTION
             '(8 7 6)
             (JOIN 5 0
                   (JOIN 0 0
                         (SELECTION-CONSTANT 2 'LUMMOX
                                             (SELECTION-CONSTANT 1 'LUMMOX
                              

## TODO-8 (10 pts): Pushing Selections into Joins

A key part of the answer to TODO-6 looked like this:

$$\sigma_{LastName, Long}(STUDENTS \bowtie_{StudentId, StudentId} TRANSCRIPT)$$

This computes the join of the two tables and then selects only the relevant rows. But what if we did it the other way around? First, select the relevant rows from the first table, then join those rows with the second table. You should convince yourself that that will return the same answer.

Now, let's use ACL2 to guarantee that this is the case. Use `test?` to verify that these two execution ideas will always return the same values:
* $\sigma_{i,x}(A \bowtie_{j,k} B)$
* ($\sigma_{i,x}A) \bowtie_{j,k} B$

> Note: In reality, of course, the two expressions will return the same values, but one of them may be much faster than the other. Which is better and how much better will depend on details about the amount of data and the way the data is organized on disk. A good relational database (think Oracle, MySQL, PostgreSQL, SQL Server, etc.) will consider many different but equivalent *query execution plans* and pick the one that they think will result in the best overall performance. This happens essentially transparently to you, and you will only have to interfere when the query execution is just too slow. When that happens, you can see the query plan that the database came up with (i.e., the relational algebra expression it chose as "the best one") and modify the database accordingly. Details are in a future course, but way beyond this one.

Hint: Take some time to think through the hypothesis. E.g., does this theorem really hold for all values of $i$, $j$, and $k$? Do you need some extra assumptions about $A$ or $B$?

Hint: No matter what, remember that all variables need to be typed explicitly in the hypotheses.

In [189]:
(defsnapshot todo-8)

(definec way1 (i :nat x :generic-value j :nat k :nat A :generic-table B :generic-table) :generic-table
    (selection-constant i x (join j k A B))
)

(definec way2 (i :nat x :generic-value j :nat k :nat A :generic-table B :generic-table) :generic-table
    (join j k (selection-constant i x A) B)
)

(test? (implies (and (natp i) (generic-valuep x) (natp j) (natp k) (generic-tablep A) (generic-tablep B))
                (equal (way1 i x j k A B) (way2 i x j k A B))))


ACL2S !>>(DEFSNAPSHOT TODO-8)

Summary
Form:  ( DEFLABEL TODO-8 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-8
ACL2S !>>(DEFINEC WAY1
                  (I NAT X GENERIC-VALUE J NAT
                     K NAT A GENERIC-TABLE B GENERIC-TABLE)
                  GENERIC-TABLE
                  (SELECTION-CONSTANT I X (JOIN J K A B)))

Form:  ( TEST-DEFINITION WAY1 ... )
Form:  ( TEST-BODY-CONTRACTS WAY1... ) 
Form:  ( TEST-FUNCTION-CONTRACT WAY1 ...) 
Testing: Done 
Elapsed Run Time: 0.39 seconds
Form:  ( ADMIT-DEFINITION WAY1 ... )
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( PROVE-FUNCTION-CONTRACT WAY1 ... )
Time:  0.17 seconds (prove: 0.14, print: 0.00, other: 0.03)
Form:  ( PROVE-BODY-CONTRACTS WAY1 ... )
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Elapsed Run Time: 0.24 seconds
Function Name : WAY1 
Termination proven -------- [*] 
Function Contract proven -- [*] 
Body Contracts proven ----- [*]
 T
ACL2S !>>

## EC-1 (20 pts): Prove It

Use `thm` to prove that the property from TODO-8 holds. Warning: This is hard.

In [187]:
(defsnapshot ec-1)
(thm (implies (and (natp i) (generic-valuep x) (natp j) (natp k) (generic-tablep A) (generic-tablep B) (not (endp x)))
                (equal (way1 i x j k A B) (way2 i x j k A B))))



ACL2S !>>(DEFSNAPSHOT EC-1)

Summary
Form:  ( DEFLABEL EC-1 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 EC-1
ACL2S !>>(THM (IMPLIES (AND (NATP I)
                            (GENERIC-VALUEP X)
                            (NATP J)
                            (NATP K)
                            (GENERIC-TABLEP A)
                            (GENERIC-TABLEP B)
                            (NOT (ENDP X)))
                       (EQUAL (WAY1 I X J K A B)
                              (WAY2 I X J K A B))))

ACL2 Warning [Invariant-risk] in ACL2::TEST-CHECKPOINT:  Invariant-
risk has been detected for a call of function ACL2::TEST-CHECKPOINT
(as possibly leading to an ill-guarded call of CGEN::UI); see :DOC
invariant-risk.


But we reduce the conjecture to T, by the :forward-chaining rule 
GENERIC-VALUE=>DEF and the :type-prescription rule GENERIC-VALUEP.

Q.E.D.

Summary
Form:  ( THM ...)
Rules: ((:FORWARD-CHAINING GENERIC-VALUE=>DEF)
        (:TYPE-PRESCRIPT